# Exemples sur Indicateurs de prix

In [10]:
from apifoncier import configure
import apifoncier.ind_prix as prix

import requests

import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import folium
import branca

pio.renderers.default = "notebook"
configure(PROGRESS_BAR=False)

### Evolution des prix et des volumes de transactions

In [15]:
df = prix.communes(code_insee="59646")

fig = make_subplots(specs=[[{"secondary_y": True}]])

palette = ["indianred", "lightsalmon"]

# Ajout des barres de nombre de transaction
fig.add_trace(
    go.Bar(
        x=df["annee"],
        y=df["nbtrans_cod1113"],
        name="Maisons anciennes",
        marker_color=palette[0],
        opacity=0.7,
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(
        x=df["annee"],
        y=df["nbtrans_cod1213"],
        name="Appartements anciens",
        marker_color=palette[1],
        opacity=0.7,
    ),
    secondary_y=False,
)

# Ajout des courbes de prix médian
fig.add_trace(
    go.Scatter(
        x=df["annee"],
        y=df["pxm2_median_cod1113"],
        mode="lines",
        name="Maisons anciennes",
        line=dict(color=palette[0]),
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df["annee"],
        y=df["pxm2_median_cod1213"],
        mode="lines",
        name="Appartements anciens",
        line=dict(color=palette[1]),
    ),
    secondary_y=True,
)

# Mise à jour de la mise en page du graphique
fig.update_layout(
    title="Évolution annuelle des prix médian et du nombre de transaction (Wasquehal)",
    xaxis_title="Année",
    yaxis_title="Nombre de ventes",
    yaxis2=dict(title="Prix médian au m2", overlaying="y", side="right"),
)

### Carte communale à l'échelle d'un EPCI

In [20]:
# On définit l'EPCI et on récupère les contours des communes qui la compose
EPCI = "245900758"
communes = requests.get(
    f"https://geo.api.gouv.fr/epcis/{EPCI}/communes/?format=geojson&geometry=contour"
).json()

# On choisit un indicateur de prix
indicateur, titre = "valeurfonc_median_mmx", "Maisons - surface entre 90 et 130 m2 "
mini, maxi = 99999999, 0

# Ajout de l'indicateur pour chaque contour de communes
for i, comm in enumerate(communes["features"]):
    code = comm["properties"]["code"]
    nom = comm["properties"]["nom"]
    print(nom, str(i + 1) + "/" + str(len(communes["features"])))
    response = prix.communes(code_insee=code, periode="triennal", annee="2019")
    if not response.empty:
        value = response[indicateur].values[0] or 0
        comm["properties"][indicateur] = value
        if value > 0:
            if value < mini:
                mini = value
            if value > maxi:
                maxi = value

# Génération de la carte
colorscale = branca.colormap.linear.YlOrRd_09.scale(mini, maxi)
colorscale.caption = "Prix médian (€) communal entre 2019 et 2021"


def style_function(feature):
    value = feature["properties"][indicateur]
    return {
        "fillOpacity": 0.8,
        "weight": 1,
        "fillColor": colorscale(int(value)) if value > 0 else "lightgray",
        "color": "darkgray",
    }


m = folium.Map(location=[50.65, 2.74], zoom_start=11)
folium.GeoJson(
    communes,
    name=titre,
    style_function=style_function,
    popup=folium.GeoJsonPopup(
        fields=["nom", indicateur], aliases=["nom", "Prix median"]
    ),
).add_to(m)
colorscale.add_to(m)
m

Estaires 1/8
La Gorgue 2/8
Haverskerque 3/8
Merville 4/8
Fleurbaix 5/8
Laventie 6/8
Lestrem 7/8
Sailly-sur-la-Lys 8/8
